In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [11]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [12]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100]

In [13]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 68,  1, 60, 68,  1, 73, 68],
        [ 9,  1, 72, 68, 66, 58,  1, 73],
        [71, 75, 58, 65, 68, 74, 72, 65],
        [67, 60,  1, 59, 62, 71, 72, 73]])
targets:
tensor([[68,  1, 60, 68,  1, 73, 68, 11],
        [ 1, 72, 68, 66, 58,  1, 73, 62],
        [75, 58, 65, 68, 74, 72, 65, 78],
        [60,  1, 59, 62, 71, 72, 73,  1]])


In [17]:
#Block after will the perdicted one, that will be added 
block_size=8 #size used for training
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('when the i/p is',context,'target is',target)

when the i/p is tensor([80]) target is tensor(1)
when the i/p is tensor([80,  1]) target is tensor(1)
when the i/p is tensor([80,  1,  1]) target is tensor(28)
when the i/p is tensor([80,  1,  1, 28]) target is tensor(39)
when the i/p is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when the i/p is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when the i/p is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when the i/p is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


EX4qYEA,yC(VBjjuaUvYCM eynxjz0t9fl,
Enyv_RhL:bx1Ek,b1ECsx"AE;zdhKO"1Fx Sq[xOPDV_zaUzh:,4
ycRI1Sdk[yS3lR"] 1"l&xw79osN4PqU.n[YW?g&6WURwEJlD.w.Qg6eycydln56P_*J:W(att!l'B T ;(pzB﻿zn[GcQsI﻿TJv?zWl]v
Vc,yCs,bG8]Fn[[PMCRI,Gb6OvD,9eF;N)AA80'(];z﻿Z9o?aUJ]]XU(tGZ]G_RLk1,aQgDhdP:QJmB
.y&?7aruwD,4I A﻿zVk﻿n[b7vYW8rQEGrMeRGQ Akt7CSj3t9E)S:FYkFG;&Z1W*aY?(!q[I;eR﻿fN4(6MR"2CG0""yh)juV-3auqr'Y6aKFh]'R'Rn8VN!A,bjQ2EC2n[8!&&nG_C)N6P,vUp(]I w;zc;zBJvhyZ5vYKRJHmgFb6﻿FD[bFg[f-lJO9OvZ&.wprAk!N_ca:M:Zqxw7vYkWcmQBJHwbxX


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.893, val loss: 4.921
step: 250, train loss: 4.852, val loss: 4.864
step: 500, train loss: 4.769, val loss: 4.822
step: 750, train loss: 4.721, val loss: 4.731
4.677820682525635


In [16]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


O_﻿8Cohh tteUs:2[n)Km*?gBJv4YW-aQ"JHYW;zd0_kig2 ?rM4ONoi?tgu'9tjuVUI[XxcR"]Q1uBpwUh[dnbxUHebT[okF HEGVI 1Eu]BxcF2c
vb?kfLwm gqOvba3Jgt(Ia;B 1)f?equOj,4leZe4G!75 ZQxSaFixQ 14If;rf!xZxbHKqpUL9?wMq"ycSd'﻿z!cG1MzL',,fUZ"cR""GNu(6_4j30*6vYQ4US7v6E.'DxUnf,j]L;,gqE]OF
)j6E,n1bxbjz8eUOF!8T7,bo&(;-ZdGG5V(ICdxWu[kbso5vNM;Y,6&pHYdWkXa'c8TJKtd8p9fdQ.w_Jm';zaa"1]9PYWfOhFAkOPat9xe5*aa
)EHYWa)Xv
ueP﻿z﻿fI37V*OPG8z6Oon4MMSioLg5-7wX:Z31-7aeP:YW*-l5YW*JH:]&wey3Xh7K,vXznPG[)b;rWX:JLlAh'p9PUlwVFDfyYWO1M,g)fI,9qCr-7u
